---

_You are currently looking at **version 1.0** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-social-network-analysis/resources/yPcBs) course resource._

---

# Assignment 4

In [1]:
import networkx as nx
import pandas as pd
import numpy as np
import pickle

---

## Part 1 - Random Graph Identification

For the first part of this assignment you will analyze randomly generated graphs and determine which algorithm created them.

In [2]:
P1_Graphs = pickle.load(open('A4_graphs','rb'))
P1_Graphs

<br>
`P1_Graphs` is a list containing 5 networkx graphs. Each of these graphs were generated by one of three possible algorithms:
* Preferential Attachment (`'PA'`)
* Small World with low probability of rewiring (`'SW_L'`)
* Small World with high probability of rewiring (`'SW_H'`)

Anaylze each of the 5 graphs and determine which of the three algorithms generated the graph.

*The `graph_identification` function should return a list of length 5 where each element in the list is either `'PA'`, `'SW_L'`, or `'SW_H'`.*

In [3]:
#G=P1_Graphs[4]
#degree=G.degree()
#degree_values=sorted(set(degree.values()))
#histogram=[list(degree.values()).count(i)/float(nx.number_of_nodes(G)) for i in degree_values]
#import matplotlib.pyplot as plt
#plt.bar(degree_values,histogram)
#plt.show()

In [4]:
def graph_identification():
    
    # Your Code Here
    
    return ['PA','SW_L','SW_L','PA','SW_H'] # Your Answer Here

---

## Part 2 - Company Emails

For the second part of this assignment you will be workking with a company's email network where each node corresponds to a person at the company, and each edge indicates that at least one email has been sent between two people.

The network also contains the node attributes `Department` and `ManagmentSalary`.

`Department` indicates the department in the company which the person belongs to, and `ManagmentSalary` indicates whether that person is receiving a managment position salary.

In [5]:
G = nx.read_gpickle('email_prediction.txt')

print(nx.info(G))

Name: 
Type: Graph
Number of nodes: 1005
Number of edges: 16706
Average degree:  33.2458


### Part 2A - Salary Prediction

Using network `G`, identify the people in the network with missing values for the node attribute `ManagementSalary` and predict whether or not these individuals are receiving a managment position salary.

To accomplish this, you will need to create a matrix of node features using networkx, train a sklearn classifier on nodes that have `ManagementSalary` data, and predict a probability of the node receiving a managment salary for nodes where `ManagementSalary` is missing.



Your predictions will need to be given as the probability that the corresponding employee is receiving a managment position salary.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.75 or higher will receive full points.

Using your trained classifier, return a series of length 252 with the data being the probability of receiving managment salary, and the index being the node id.

    Example:
    
        1       1.0
        2       0.0
        5       0.8
        8       1.0
            ...
        996     0.7
        1000    0.5
        1001    0.0
        Length: 252, dtype: float64

In [6]:
df=pd.DataFrame(index=G.nodes())
df['Department']=pd.Series(nx.get_node_attributes(G,'Department'))
df['ManagementSalary']=pd.Series(nx.get_node_attributes(G,'ManagementSalary'))
df['degree']=pd.Series(G.degree())
df['clustering']=pd.Series(nx.clustering(G))
df['closeCent']=pd.Series(nx.closeness_centrality(G))
df['btwCent']=pd.Series(nx.betweenness_centrality(G,normalized=True,endpoints=False))
df_train=df[df['ManagementSalary'].notnull()][['Department','degree','clustering','closeCent','btwCent']]
df_train_target=df[df['ManagementSalary'].notnull()]['ManagementSalary']

In [7]:
def salary_predictions():
    from sklearn.ensemble import RandomForestClassifier
    # Your Code Here
    df=pd.DataFrame(index=G.nodes())
    df['Department']=pd.Series(nx.get_node_attributes(G,'Department'))
    df['ManagementSalary']=pd.Series(nx.get_node_attributes(G,'ManagementSalary'))
    df['degree']=pd.Series(G.degree())
    df['clustering']=pd.Series(nx.clustering(G))
    df['closeCent']=pd.Series(nx.closeness_centrality(G))
    df['btwCent']=pd.Series(nx.betweenness_centrality(G,normalized=True,endpoints=False))
    df_train=df[df['ManagementSalary'].notnull()][['Department','degree','clustering','closeCent','btwCent']]
    df_train_target=df[df['ManagementSalary'].notnull()]['ManagementSalary']
    df_test=df[df['ManagementSalary'].isnull()][['Department','degree','clustering','closeCent','btwCent']]
    
    rfc=RandomForestClassifier(n_estimators=100,max_depth=4)
    rfc.fit(df_train,df_train_target)
    
    prob=rfc.predict_proba(df_test)
    probs=pd.Series([i[1] for i in prob],index=df_test.index)
    return probs # Your Answer Here

In [8]:
salary_predictions()

1       0.079912
2       0.912221
5       0.998530
8       0.209975
14      0.119954
18      0.102592
27      0.128552
30      0.431220
31      0.258625
34      0.081265
37      0.122787
40      0.101526
45      0.079588
54      0.314625
55      0.449756
60      0.212431
62      0.996863
65      0.910892
77      0.045489
79      0.124040
97      0.033509
101     0.027819
103     0.430840
108     0.121626
113     0.211521
122     0.023392
141     0.324791
142     0.996863
144     0.040637
145     0.411162
          ...   
913     0.043658
914     0.028261
915     0.008390
918     0.060521
923     0.045276
926     0.063152
931     0.025848
934     0.012040
939     0.008680
944     0.008548
945     0.045276
947     0.063448
950     0.156562
951     0.025095
953     0.015445
959     0.007300
962     0.007111
963     0.206413
968     0.057973
969     0.061226
974     0.045168
984     0.012118
987     0.054161
989     0.060948
991     0.073495
992     0.007300
994     0.008656
996     0.0083

### Part 2B - New Connections Prediction

For the last part of this assignment, you will predict future connections between employees of the network. The future connections information has been loaded into the variable `future_connections`. The index is a tuple indicating a pair of nodes that currently do not have a connection, and the `Future Connection` column indicates if an edge between those two nodes will exist in the future, where a value of 1.0 indicates a future connection.

In [9]:
future_connections = pd.read_csv('Future_Connections.csv', index_col=0, converters={0: eval})
future_connections.head(10)

,Future Connection
"(6, 840)",0.0
"(4, 197)",0.0
"(620, 979)",0.0
"(519, 872)",0.0
"(382, 423)",0.0
"(97, 226)",1.0
"(349, 905)",0.0
"(429, 860)",0.0
"(309, 989)",0.0
"(468, 880)",0.0


Using network `G` and `future_connections`, identify the edges in `future_connections` with missing values and predict whether or not these edges will have a future connection.

To accomplish this, you will need to create a matrix of features for the edges found in `future_connections` using networkx, train a sklearn classifier on those edges in `future_connections` that have `Future Connection` data, and predict a probability of the edge being a future connection for those edges in `future_connections` where `Future Connection` is missing.



Your predictions will need to be given as the probability of the corresponding edge being a future connection.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.75 or higher will receive full points.

Using your trained classifier, return a series of length 122112 with the data being the probability of the edge being a future connection, and the index being the edge as represented by a tuple of nodes.

    Example:
    
        (107, 348)    0.35
        (542, 751)    0.40
        (20, 426)     0.55
        (50, 989)     0.35
                  ...
        (939, 940)    0.15
        (555, 905)    0.35
        (75, 101)     0.65
        Length: 122112, dtype: float64

In [12]:
def new_connections_predictions():
    from sklearn.ensemble import RandomForestClassifier
    # Your Code Here
    df_edge = future_connections
    df_edge['pref_attachment'] = [i[2] for i in nx.preferential_attachment(G, df_edge.index)]
    df_edge['common_neighbors'] = df_edge.index.map(lambda edge: len(list(nx.common_neighbors(G, edge[0], edge[1]))))
    df_edge['jacc_coeff'] = [i[2] for i in nx.jaccard_coefficient(G, df_edge.index)]
    df_edge['resource_alloc'] = [i[2] for i in nx.resource_allocation_index(G, df_edge.index)]
    df_edge['adamic_adar'] = [i[2] for i in nx.adamic_adar_index(G, df_edge.index)]
    
    df_edge_train=df_edge[df_edge['Future Connection'].notnull()][['pref_attachment','common_neighbors','jacc_coeff','resource_alloc','adamic_adar']]
    df_edge_target=df_edge[df_edge['Future Connection'].notnull()]['Future Connection']
    
    df_edge_test=df_edge[df_edge['Future Connection'].isnull()][['pref_attachment','common_neighbors','jacc_coeff','resource_alloc','adamic_adar']]
    
    rfc=RandomForestClassifier(n_estimators=100,max_depth=5)
    rfc.fit(df_edge_train,df_edge_target)
    
    prob=rfc.predict_proba(df_edge_test)
    probs=pd.Series([i[1] for i in prob],index=df_edge_test.index)
    return probs # Your Answer Here

In [13]:
new_connections_predictions()

(107, 348)    0.034245
(542, 751)    0.013079
(20, 426)     0.552524
(50, 989)     0.013079
(942, 986)    0.013079
(324, 857)    0.013079
(13, 710)     0.183319
(19, 271)     0.143366
(319, 878)    0.013079
(659, 707)    0.013079
(49, 843)     0.013079
(208, 893)    0.013079
(377, 469)    0.013137
(405, 999)    0.020472
(129, 740)    0.014344
(292, 618)    0.052598
(239, 689)    0.013079
(359, 373)    0.013137
(53, 523)     0.032916
(276, 984)    0.013079
(202, 997)    0.013079
(604, 619)    0.078015
(270, 911)    0.013079
(261, 481)    0.067704
(200, 450)    0.894600
(213, 634)    0.013079
(644, 735)    0.149128
(346, 553)    0.013079
(521, 738)    0.013079
(422, 953)    0.020389
                ...   
(672, 848)    0.013079
(28, 127)     0.968203
(202, 661)    0.013079
(54, 195)     0.996666
(295, 864)    0.013079
(814, 936)    0.013079
(839, 874)    0.013079
(139, 843)    0.013079
(461, 544)    0.013079
(68, 487)     0.013079
(622, 932)    0.013079
(504, 936)    0.016367
(479, 528) 